# Totten Ice Shelf geometry

In [ ]:
import sys
import cartopy
import shapefile
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib
import cmocean.cm as cmo
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

sys.path.append("..")
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

## Bedmachine topography data

In [ ]:
ds = xr.open_dataset('../../../../Downloads/BedMachineAntarctica_2019-11-05_v01.nc')

In [ ]:
ds

In [ ]:
%%time
plt.figure(figsize=(20,20))
(ds.surface-ds.thickness).where(ds.mask==3).plot()

### selecting Totten Ice Shelf

In [ ]:
xlim = slice(2.19e6,2.322e6)
ylim = slice(-1e6,-1.3e6)
lim = {'x':xlim, 'y':ylim}

In [ ]:
mask = ds['mask'].sel(lim)
mask.plot()

### create grounding line and ice front masks

In [ ]:
grl = xr.where(mask-mask.shift(x= 1)==1, mask, 0) + \
      xr.where(mask-mask.shift(x=-1)==1, mask, 0) + \
      xr.where(mask-mask.shift(y= 1)==1, mask, 0) + \
      xr.where(mask-mask.shift(y=-1)==1, mask, 0)
grl = grl/grl
grl.name = 'grounding_line'

isf = xr.where(mask-mask.shift(x= 1)==3, mask, 0) + \
      xr.where(mask-mask.shift(x=-1)==3, mask, 0) + \
      xr.where(mask-mask.shift(y= 1)==3, mask, 0) + \
      xr.where(mask-mask.shift(y=-1)==3, mask, 0)
isf = isf/isf
isf.name = 'iceshelf_front'

In [ ]:
plt.figure(figsize=(10,10), constrained_layout=True)
# (ds.surface-ds.thickness).where(ds.mask==3).where(ds.surface>0).sel(lim).plot(**kwargs)
grl.where(grl>0).plot(cmap='Blues', add_colorbar=False)
isf.where(isf>0).plot(cmap='Reds' , add_colorbar=False)

In [ ]:
X, Y = np.meshgrid(mask.x.values, mask.y.values)
x = xr.DataArray(data=X, dims=['y','x'], coords={'y':mask.y,'x':mask.x})
y = xr.DataArray(data=Y, dims=['y','x'], coords={'y':mask.y,'x':mask.x})

mask_a = xr.where(mask==3, 1, 0)
mask_b = xr.where(grl ==1, 1, 0)
mask_c = xr.where(isf ==1, 1, 0)

In [ ]:
def distance_to_line(x,y,mask_a,mask_b):
    """ calculate minimum distance from all points in mask_a to points in mask_b
    input:  (all 2D arrays)
    x, y    ..  x/y coordinate xr.DataArrays
    mask_a  ..  mask of points for which minimum distance to mask_b is determined
    mask_b  ..  mask of line (grounding line/ ice shelf front)
    
    output:  reconstructed xr.DataArray with distances
    """
    # stacking into single dimension
    stackkws = {'all_points':['x','y']}
    x_ = x.stack(**stackkws)
    y_ = y.stack(**stackkws)
    mask_a_ = mask_a.stack(**stackkws)
    mask_b_ = mask_b.stack(**stackkws)
    
    # masking both x,y by both masks to reduce computational load
    ma_x = x_.where(mask_a_).dropna(dim='all_points')
    ma_y = y_.where(mask_a_).dropna(dim='all_points')
    mb_x = x_.where(mask_b_).dropna(dim='all_points')
    mb_y = y_.where(mask_b_).dropna(dim='all_points')
    index = pd.MultiIndex.from_tuples(list(zip(*[ma_y.values,ma_x.values])),names=['y','x'])
    Na, Nb = len(ma_x.values), len(mb_x.values)
    # to indicate cost savings
    print(f'number of points in mask_a: {Na:6d} ; percentage of total array points: {Na/len(x_)*100:5.2f} %')
    print(f'number of points in mask_b: {Nb:6d} ; percentage of total array points: {Nb/len(x_)*100:5.2f} %')
    
    # calculate euclidean distance and find minimum                   
    dist = np.min(np.sqrt((np.tile(ma_x.values,(Nb,1)) - np.tile(mb_x.values.reshape(-1,1),Na))**2 + 
                          (np.tile(ma_y.values,(Nb,1)) - np.tile(mb_y.values.reshape(-1,1),Na))**2), axis=0)
    s = pd.Series(dist, index=index)
    return xr.DataArray.from_series(s)

In [ ]:
%%time
# distance to grounding line
di = distance_to_line(x,y,mask_a,mask_c)
dg = distance_to_line(x,y,mask_a,mask_b)

In [ ]:
divnorm = matplotlib.colors.DivergingNorm(vmin=-3000., vcenter=0, vmax=500)
kwargs = {'cbar_kwargs':{'orientation':'horizontal'}}
f, ax = plt.subplots(2, 3, figsize=(15,10), constrained_layout=True, sharey=True, sharex=True)

ds.surface.where(ds.surface>0).sel(lim).plot(ax=ax[0,0], **kwargs)
ds.bed.sel(lim).plot(ax=ax[0,1], cmap=cmo.balance, norm=divnorm, **kwargs)
ds.thickness.where(ds.thickness>0).sel(lim).plot(ax=ax[0,2], cmap='magma_r', **kwargs)
da_ = (ds.surface-ds.thickness).where(ds.mask==3).sel(lim)
da_.name = 'draft [meters]'
da_.plot(ax=ax[1,0], cmap='plasma', **kwargs, vmax=0)
grl.where(grl>0).plot(ax=ax[1,0], cmap='Blues', add_colorbar=False)
isf.where(isf>0).plot(ax=ax[1,0], cmap='Reds'   , add_colorbar=False)
di.name = 'to ice shelf front [km]'
(di/1e3).plot(ax=ax[1,1], **kwargs)
dg.name = 'to grounding line [km]'
(dg/1e3).plot(ax=ax[1,2], **kwargs)

f.suptitle('Totten Ice Shelf', fontsize=18)
plt.savefig('../../results/Bedmachine/TottenIS_geometry')

## projection of bed data

In [ ]:
colors_undersea = cmo.deep_r(np.linspace(0, 1, 256))
colors_land = matplotlib.cm.cividis_r(np.linspace(0.1, .5, 256))
all_colors = np.vstack((colors_undersea, colors_land))
terrain_map = matplotlib.colors.LinearSegmentedColormap.from_list('terrain_map', all_colors)
divnorm = matplotlib.colors.DivergingNorm(vmin=-6000., vcenter=0, vmax=2000)

# example plot
X = np.random.rand(5,5)*4000-2000
plt.pcolormesh(X, cmap=terrain_map, norm=divnorm)
plt.colorbar()

In [ ]:
%%time
plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=-90, central_longitude=0))
ax.pcolormesh(ds.x,ds.y,ds.bed, cmap=terrain_map, norm=divnorm,
              transform=ccrs.SouthPolarStereo(true_scale_latitude=-71),
             )
ax.coastlines()
ax.gridlines()

## BAS shapefile

In [ ]:
# shapefile
sfn = '../../../../Downloads/add_coastline_high_res_polygon_v7.2/add_coastline_high_res_polygon_v7.2.shp'
sfr = cartopy.io.shapereader.Reader(sfn)
sfr_ = shapefile.Reader(sfn)
# sfr_.shapeType # polygons
len(sfr_.shapes())
sfr_.records()

In [ ]:
%%time
plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.SouthPolarStereo(true_scale_latitude=71))  # latitude mentioned here: https://epsg.io/3031
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
shape_feature = cartopy.feature.ShapelyFeature(sfr.geometries(),
                                               ccrs.SouthPolarStereo(true_scale_latitude=71), edgecolor='black')
ax.add_feature(shape_feature, edgecolor='r')
ax.coastlines()
ax.gridlines()

# theta = np.linspace(0, 2*np.pi, 100)
# center, radius = [0.5, 0.5], 0.5
# verts = np.vstack([np.sin(theta), np.cos(theta)]).T
# circle = mpath.Path(verts * radius + center)

# ax2.set_boundary(circle, transform=ax2.transAxes)
